In [1]:
from scipy.optimize import root
import numpy as np

In [2]:
params = {
    'eps': 1.5,
    'tht': 1,
    'alf': .5,
    'bet': .9,
    'om_0': 1,
    'om_1': 1,
    'om_2': 1,
    'delt': .05
}

In [3]:
ss_shock = {
    'a_0': 0,
    'a_1': 0,
    'a_2': 0
}

In [4]:
shock_params = {
    'var_a_0': .5,
    'var_a_1': .6,
    'var_a_2': .7,
    'rho_a_0': .9,
    'rho_a_1': .9,
    'rho_a_2': .9
}

In [5]:
ss_params = {**params, **ss_shock}
model_params = {**params, **shock_params}

In [6]:
def assign_params(ss_params):
    a_0 = ss_params['a_0']
    a_1 = ss_params['a_1']
    a_2 = ss_params['a_2']
    alf = ss_params['alf']
    delt = ss_params['delt']
    om_0 = ss_params['om_0']
    om_1 = ss_params['om_1']
    om_2 = ss_params['om_2']
    eps = ss_params['eps']
    tht = ss_params['tht']
    bet = ss_params['bet']
   
    return eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2


In [7]:
n_goods=3

def write_dict_to_file(filename):
    
    with open(filename, 'w') as f:
        
        
        f.write('def equations(vars, params):\n')
        f.write('\tn_0,n_1,n_2,k_0,k_1,k_2 = vars\n')
        f.write('\teps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2 = assign_params(params)\n')
    
    
    
    
    
        
        f.write('\tph=(')
        for i in range(0, n_goods):
            f.write(f'\tom_{i}*c_{i}**((eps-1)/eps)')
            if i < n_goods-1:
                f.write(f'+')
            f.write('')    
        f.write('\t)**((1-tht*eps)/(eps-1))\n')

        f.write(f'\n')
        for i in range(0, n_goods):
            f.write(f'\tv_{i}=om_{i}*c_{i}**(-1/eps)*ph\n')
        

        f.write(f'\n')
        for i in range(0, n_goods):
            f.write(f'\tfk_{i}=exp(a_{i})*n_{i}*alf*k{i}**(alf-1)\n')
        
        
        f.write(f'\n')
        for i in range(0, n_goods):
            f.write(f'\tfn_{i}=a_{i}*(1-alf)*k{i}**(alf)\n')
        
        f.write(f'\n')
        f.write(f'\tlamk=v_0*fk_0/n_0\n')
        
        
        
        f.write(f'\teq1=v_1*fk_1-n_1*lamk\n')
        f.write(f'\teq2=v_2*fk_2-n_2*lamk\n')
        f.write(f'\n')
        f.write(f'\teq3=bet*(fk_0+1-delt)-1\n')
        f.write(f'\teq4=n_0+n_1+n_2-1\n')
        f.write(f'\teq5=c_0-exp(a_0)*n_0*k**alf-(1-delt)*k+k\n')
        f.write(f'\teq6=c_1-exp(a_1)*n_1*k**alf\n')
        f.write(f'\teq7=c_2-exp(a_2)*n_2*k**alf\n')

        f.write(f'\treturn [eq1, eq2, eq3, eq4, eq5, eq6, eq7]\n')

               
# Call the function
write_dict_to_file('t_ss_model.m')


In [8]:
%run t_ss_model.m

In [9]:
def equations(vars, params):
	n_0,n_1,n_2,k,c_0,c_1,c_2 = vars
	eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2 = assign_params(params)
	ph=(	om_0*c_0**((eps-1)/eps)+	om_1*c_1**((eps-1)/eps)+	om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))

	v_0=om_0*c_0**(-1/eps)*ph
	v_1=om_1*c_1**(-1/eps)*ph
	v_2=om_2*c_2**(-1/eps)*ph

	fk_0=np.exp(a_0)*n_0*alf*k**(alf-1)
	fk_1=np.exp(a_1)*n_1*alf*k**(alf-1)
	fk_2=np.exp(a_2)*n_2*alf*k**(alf-1)

	fn_0=a_0*(1-alf)*k**(alf)
	fn_1=a_1*(1-alf)*k**(alf)
	fn_2=a_2*(1-alf)*k**(alf)

	lamk=v_0*fk_0/n_0
	eq1=v_1*fk_1-n_1*lamk
	eq2=v_2*fk_2-n_2*lamk

	eq3=bet*(fk_0+1-delt)-1
	eq4=n_0+n_1+n_2-1
	eq5=c_0-np.exp(a_0)*n_0*k**alf-(1-delt)*k+k
	eq6=c_1-np.exp(a_1)*n_1*k**alf
	eq7=c_2-np.exp(a_2)*n_2*k**alf
	return [eq1, eq2, eq3, eq4, eq5, eq6, eq7]

In [10]:
initial_guess = np.array([.3, .3, .3, .3, .3, .3,.3])

In [11]:
sol = root(equations, initial_guess, args=(ss_params,))

In [12]:
sol.x

array([0.37179487, 0.31410256, 0.31410256, 1.33136095, 0.36242604,
       0.36242604, 0.36242604])